In [1]:
import sys
path = './code'
sys.path.insert(0, path)

In [2]:
import cv2, torch
import numpy as np
from os.path import realpath, dirname, join

from net import SiamRPNvot, SiamRPNBIG, SiamRPNotb
from run_SiamRPN import SiamRPN_init, SiamRPN_track
from utils import get_axis_aligned_bbox, cxy_wh_2_rect

# load net
# net = SiamRPNvot()
# net.load_state_dict(torch.load('SiamRPNVOT.model'))

# net = SiamRPNBIG()
# net.load_state_dict(torch.load('SiamRPNBIG.model'))

net = SiamRPNotb()
net.load_state_dict(torch.load('SiamRPNOTB.model'))

net.eval().cuda()

SiamRPNotb(
  (featureExtract): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(2, 2))
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU(inplace)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): ReLU(inplace)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1))
    (9): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace)
    (11): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1))
    (12): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace)
    (14): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1))
    (15): BatchNorm2d(256, ep

In [3]:
# function for select target
def mouse_opt(event, x, y, flags, param):
    global target_pos, target_sz, tmp_p_1, tmp_p_2
    if event == cv2.EVENT_LBUTTONDOWN:
        tmp_p_1 = (x, y)
    elif event == cv2.EVENT_LBUTTONUP:
        tmp_p_2 = (x, y)
        # update target bounding box
        x_c = (tmp_p_1[0] + tmp_p_2[0]) / 2.0
        y_c = (tmp_p_1[1] + tmp_p_2[1]) / 2.0
        w = float(abs(tmp_p_2[0] - tmp_p_1[0]))
        h = float(abs(tmp_p_2[1] - tmp_p_1[1]))
        target_pos = np.array([x_c, y_c])
        target_sz = np.array([w, h])
        print('target selected at center: (%d, %d) and size: (%d, %d)' % (x_c, y_c, w, h))

def draw():
    global im_show, target_pos, target_sz
    im_show = im.copy()
    
    if target_pos is not None:
        cv2.rectangle(im_show, (int(target_pos[0] - target_sz[0] / 2), int(target_pos[1] - target_sz[1] / 2)),
                      (int(target_pos[0] + target_sz[0] / 2), int(target_pos[1] + target_sz[1] / 2)),
                      (0, 255, 0), 1)

#### select target

In [8]:
video = cv2.VideoCapture("test.mp4")
_, im = video.read()

global target_pos, target_sz, tmp_p_1, tmp_p_2
tmp_p_1 = (0, 0)
tmp_p_2 = (0, 0)
target_pos = None
target_sz = None

global im_show
im_show = im.copy()
while True:
    cv2.setMouseCallback("img", mouse_opt)
    draw()
    cv2.imshow("img", im_show)
    k = cv2.waitKey(1)
    if k == 27:
        break
cv2.destroyAllWindows()

target selected at center: (396, 141) and size: (90, 95)


In [9]:
state = SiamRPN_init(im, target_pos, target_sz, net)

video = cv2.VideoCapture("test.mp4")
# tracking and visualization
toc = 0
i = 1
while True:
    ok, im = video.read()
#     im = cv2.resize(im, (960, 540))
    if not ok:
        break
    tic = cv2.getTickCount()
    state = SiamRPN_track(state, im)  # track
    toc += cv2.getTickCount()-tic
    i += 1
    res = cxy_wh_2_rect(state['target_pos'], state['target_sz'])
    res = [int(l) for l in res]
    cv2.rectangle(im, (res[0], res[1]), (res[0] + res[2], res[1] + res[3]), (0, 255, 255), 3)
    cv2.putText(im, "score: %.2f" % state['score'], (int(state['target_pos'][0]), int(state['target_pos'][1])), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 25, 255), 2)
    cv2.imshow('SiamRPN', im)
    k = cv2.waitKey(1)
    if k == 27:
        break
cv2.destroyAllWindows()
print('Tracking Speed {:.1f}fps'.format((i-1)/(toc/cv2.getTickFrequency())))

Tracking Speed 54.0fps
